In [2]:
import jsonlines
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pyspark import SparkContext
import numpy as np
import json

In [4]:
climate_videos_path = "data/refine_filtered_videos.jsonl"
climate_videos_df = pd.read_json(climate_videos_path, lines=True)

In [9]:
import numpy as np

climate_videos_df['like_ratio'] = np.where(climate_videos_df['dislike_count'] == 0, None, climate_videos_df['like_count'] / climate_videos_df['dislike_count'])


In [11]:
display(climate_videos_df['like_ratio'].sort_values(ascending=False).head(10))

93953     1039.0
112470    1015.0
27075      918.5
33291      846.0
81891      791.0
58432      756.0
98034      756.0
40348      735.0
78157      724.0
104971     704.0
Name: like_ratio, dtype: object